# Compliance Check PoC

In [19]:
import os
import openai

print(f'Check env: Key="{os.getenv("BEACON_AZURE_OPENAI_API_KEY")}"')

openai.api_key = os.getenv("BEACON_AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv("BEACON_AZURE_OPENAI_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future


from langchain.chat_models import AzureChatOpenAI
chat = AzureChatOpenAI(
    openai_api_key=openai.api_key,
    openai_api_type=openai.api_type,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-4",
    max_tokens=200,
    temperature=0.0,
    verbose=True,
)
# chat

Check env: Key="79de100f498d4cf4a2b5df513a20948a"


AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='79de100f498d4cf4a2b5df513a20948a', openai_api_base='https://mobile-beacon.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='gpt-4', model_version='', openai_api_type='azure', openai_api_version='2023-05-15')

## Understanding a Jira ticket in XML format

In [3]:
# read the exported JIRA xml
with open("./data/compliance-poc/jira/GP-6.xml", "r") as file:
  ticket_xml = file.readlines()
# ticket_xml

### Using OpenAI APIs

In [15]:
deployment_name='text-davinci-002' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

# # Send a completion call to generate an answer
# print('Sending a test completion job')
# start_phrase = 'Write a tagline for an ice cream shop. '
# response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
# text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
# print(start_phrase+text)

start_phrase = f"""\
Your're a project manager. For the given JIRA ticket delimited by triple backticks. Extract the following infromation: \
purpose of the ticket: A brief description of the intention of the given ticket.\
the assignee's email address, labels, and status of the ticekt.\
and format your answer in JSON.\
JIRA ticket: '''{ticket_xml}'''
"""
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=100)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text)

Your're a project manager. For the given JIRA ticket delimited by triple backticks. Extract the following infromation: purpose of the ticket: A brief description of the intention of the given ticket.the assignee's email address, labels, and status of the ticekt.and format your answer in JSON.JIRA ticket: '''['<!-- \n', 'RSS generated by JIRA (1001.0.0-SNAPSHOT#100234-sha1:11a23c5a4a0a5cb65b2dd7ab354b33ed75ed5fd1) at Sat Aug 19 03:16:29 UTC 2023\n', '\n', "It is possible to restrict the fields that are returned in this document by specifying the 'field' parameter in your request.\n", 'For example, to request only the issue key and summary add field=key&field=summary to the URL of your request.\n', '-->\n', '<rss version="0.92" >\n', '<channel>\n', '    <title>Jira</title>\n', '    <link>https://xinthink.atlassian.net</link>\n', '    <description>This file is an XML representation of an issue</description>\n', '    <language>en-us</language>    <build-info>\n', '        <version>1001.0.0

### Using LangChain

In [ ]:
# !pip install langchain

In [47]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

# System message?
# Your're a project manager. 

template_string = """\
For the given JIRA ticket delimited by triple backticks. Extract the following infromation:

summary of the ticket: A short description of the intention of rasing this ticket.

the assignee, labels, and status of the ticekt.

a rating of clarity of the ticket.

JIRA ticket: '''{ticket_xml}'''

{format_instructions}
"""

summary_schema = ResponseSchema(name="summary",
                                description="A brief description of this ticket, including \
                                objectives of the ticket, \
                                expected outputs from it.")
assignee_schema = ResponseSchema(name="assignee",
                                 description="A dictionary about the assignee of the ticket, including \
                                 a fullname, and an email address.")
labels_schema = ResponseSchema(name="labels",
                               description="A list of labels assigned to the ticket.")
status_schema = ResponseSchema(name="status",
                               description="The status of the ticket")
clarity_schema = ResponseSchema(name="clarity",
                                description="Rate from 0 to 5 on the clarity the ticket. \
                                Consider the following factors: \
                                1. Does it contain detailed requirements and objectives? \
                                2. Dose it define any key output from this task?")
clarity_reason_schema = ResponseSchema(name="clarity_desc",
                                       description="Describe the reason why you'd give the clarity rating.")

response_schemas = [summary_schema,
                    assignee_schema,
                    labels_schema,
                    status_schema,
                    clarity_schema,
                    clarity_reason_schema,
                   ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

prompt_template = ChatPromptTemplate.from_template(template_string)
# prompt_template

prompt = prompt_template.format_messages(ticket_xml=ticket_xml,
                                         format_instructions=format_instructions)
print(prompt)
response = chat(prompt)
# response.content

output_dict = output_parser.parse(response.content)
output_dict

[HumanMessage(content='For the given JIRA ticket delimited by triple backticks. Extract the following infromation:\n\nsummary of the ticket: A short description of the intention of rasing this ticket.\n\nthe assignee, labels, and status of the ticekt.\n\na rating of clarity of the ticket.\n\nJIRA ticket: \'\'\'[\'<!-- \\n\', \'RSS generated by JIRA (1001.0.0-SNAPSHOT#100234-sha1:11a23c5a4a0a5cb65b2dd7ab354b33ed75ed5fd1) at Sat Aug 19 03:16:29 UTC 2023\\n\', \'\\n\', "It is possible to restrict the fields that are returned in this document by specifying the \'field\' parameter in your request.\\n", \'For example, to request only the issue key and summary add field=key&field=summary to the URL of your request.\\n\', \'-->\\n\', \'<rss version="0.92" >\\n\', \'<channel>\\n\', \'    <title>Jira</title>\\n\', \'    <link>https://xinthink.atlassian.net</link>\\n\', \'    <description>This file is an XML representation of an issue</description>\\n\', \'    <language>en-us</language>    <build

{'summary': 'Compliance Check with AI PoC',
 'assignee': 'Yingxin Wu',
 'labels': ['LLM', 'PoC'],
 'status': 'In Progress',
 'clarity': '3',
 'clarity_desc': 'The ticket has a clear title and assignee, but lacks a detailed description of the task. It does not define any key output from this task.'}